## Preprocessing with scikit learn

In [ ]:
import os
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Loading file from path
def loading_file():
    file_dir = '/home/nbuser/library/1. Classifier/3. Exploratory Data Analysis'        
    file_list = glob.glob(file_dir + '/*.csv')
    csv_file = file_list[1]
    return csv_file

# Import file imto Pandas DataFrame
def importing_file(csv_file):
    df = pd.read_csv(csv_file, sep=",")
    return df

# Saving path
def saving_file(file, file_name, save_dir):
    file.to_csv(os.path.join(save_dir,file_name))


In [3]:
# Importing file + Loading  file
news_df = importing_file(loading_file())

# Top 5 records
news_df.head()

,file_name,title,news_text,category
0,047.txt,Berlin applauds Hotel Rwanda,Political thriller Hotel Rwanda was given a ro...,entertainment
1,355.txt,Wales get Williams fitness boost,Wales are hopeful that openside flanker Martyn...,sports
2,481.txt,Christmas sales worst since 1981,"UK retail sales fell in December, failing to m...",business
3,376.txt,MPs demand 'Budget leak' answers,Ministers have been asked to explain how Budge...,politics
4,044.txt,US actor Ossie Davis found dead,US actor Ossie Davis has been found dead at th...,entertainment


In [4]:
news_df.shape

(2003, 4)

In [7]:
news_df.category.value_counts()

sports           463
business         454
politics         368
tech             365
entertainment    353
Name: category, dtype: int64

### Clean the news_text column

In [8]:
import re
def clean_text(x):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', str(x))
    return text

def clean_all_text(df):
    for index, item in df.iterrows():
        cleantext = clean_text(item['news_text'])
        item['news_text'] = cleantext
    return df

In [9]:
news_df = clean_all_text(news_df)

### Split the dataset

In [10]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Create a series to store the labels: y
y = news_df["category"]

# Create training and test sets
X_train, X_test, y_train, y_test = train_test_split(news_df["news_text"], y, test_size=0.3, random_state=53)


### CountVectorizer for text classification

In [11]:
# Import the necessary modules
from sklearn.feature_extraction.text import CountVectorizer

# Initialize a CountVectorizer object: count_vectorizer
count_vectorizer = CountVectorizer(stop_words="english")

# Transform the training data using only the 'text' column values: count_train 
count_train = count_vectorizer.fit_transform(X_train)

# Transform the test data using only the 'text' column values: count_test 
count_test = count_vectorizer.transform(X_test)

            
# Print the first 10 features of the count_vectorizer
print(count_vectorizer.get_feature_names()[:100])


['00', '0001', '00051', '002', '003', '004secs', '007', '01', '0100', '0130', '019secs', '02', '0227', '024', '025', '027', '028', '03', '030', '0300', '0305', '04', '0400', '041', '0469', '05', '050505', '053', '053bn', '05overall', '06', '0605', '0619', '067', '07', '0700', '0708', '0710', '0730', '08', '0800', '083mph', '0845', '0870', '09', '090', '0900', '0950', '099', '10', '100', '1000', '10000', '100000', '1000000000', '100000ayear', '10000m', '10000vote', '1000m', '1000th', '1001st', '1008', '100bn', '100m', '100s', '101', '101115', '1012', '1013', '10137', '1015', '1019', '101yearold', '102', '1020', '1022', '10227', '1025', '10255', '1025m', '10280', '102975', '102inch', '102m', '103', '1030', '10305', '103500', '1038am', '103bn', '103m', '104', '104000', '10416', '1044', '10476', '1048', '104bn', '104m', '105']


#### TfidfVectorizer for text classification

In [12]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize a TfidfVectorizer object: tfidf_vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words="english",max_df=0.7)

# Transform the training data: tfidf_train 
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data: tfidf_test 
tfidf_test = tfidf_vectorizer.transform(X_test)
    
print(tfidf_vectorizer.get_feature_names()[:100])

['00', '0001', '00051', '002', '003', '004secs', '007', '01', '0100', '0130', '019secs', '02', '0227', '024', '025', '027', '028', '03', '030', '0300', '0305', '04', '0400', '041', '0469', '05', '050505', '053', '053bn', '05overall', '06', '0605', '0619', '067', '07', '0700', '0708', '0710', '0730', '08', '0800', '083mph', '0845', '0870', '09', '090', '0900', '0950', '099', '10', '100', '1000', '10000', '100000', '1000000000', '100000ayear', '10000m', '10000vote', '1000m', '1000th', '1001st', '1008', '100bn', '100m', '100s', '101', '101115', '1012', '1013', '10137', '1015', '1019', '101yearold', '102', '1020', '1022', '10227', '1025', '10255', '1025m', '10280', '102975', '102inch', '102m', '103', '1030', '10305', '103500', '1038am', '103bn', '103m', '104', '104000', '10416', '1044', '10476', '1048', '104bn', '104m', '105']


#### Show Vectors as Features

In [13]:
# Create the CountVectorizer DataFrame: count_df
count_df = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names())

# Create the TfidfVectorizer DataFrame: tfidf_df
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())

# Print the head of count_df
print(count_df.head())
print()
# Print the head of tfidf_df
print(tfidf_df.head())

# Calculate the difference in columns: difference
difference = set(count_df.columns) - set(tfidf_df.columns)
print(difference)

# Check whether the DataFrames are equal
print(count_df.equals(tfidf_df))


   00  0001  00051  002  003  004secs  007  01  0100  0130     ...       \
0   0     0      0    0    0        0    0   0     0     0     ...        
1   0     0      0    0    0        0    0   0     0     0     ...        
2   0     0      0    0    0        0    0   0     0     0     ...        
3   0     0      0    0    0        0    0   0     0     0     ...        
4   0     0      0    0    0        0    0   0     0     0     ...        

   zombies  zombiethe  zone  zonealarm  zones  zoom  zooropa  zubair  zurich  \
0        0          0     0          0      0     0        0       0       0   
1        0          0     0          0      0     0        0       0       0   
2        0          0     0          0      0     0        0       0       0   
3        0          0     0          0      0     0        0       0       0   
4        0          0     0          0      0     0        0       0       0   

   zvyagintsev  
0            0  
1            0  
2            0  


#### Training and testing models with CountVectorizer

In [47]:
# Import the necessary modules
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

# Instantiate a Multinomial Naive Bayes classifier: nb_classifier
nb_cv_classifier = MultinomialNB()

# Fit the classifier to the training data
nb_cv_classifier.fit(count_train, y_train)

# Create the predicted tags: pred
pred = nb_cv_classifier.predict(count_test)

# Calculate the accuracy score: score
score = metrics.accuracy_score(y_test, pred)
print(score)

# Calculate the confusion matrix: cm
cm = metrics.confusion_matrix(y_test, pred, labels=['entertainment', 'politics', 'business', 'sports', 'tech'])
print(cm)


0.9816971713810316
[[121   1   0   0   3]
 [  0 105   1   0   0]
 [  0   2 131   0   2]
 [  0   1   0 127   0]
 [  0   1   0   0 106]]


#### Training and testing the model with TfidfVectorizer

In [48]:
# Create a Multinomial Naive Bayes classifier: nb_classifier
nb_tfidf_classifier = MultinomialNB()

# Fit the classifier to the training data
nb_tfidf_classifier.fit(tfidf_train, y_train)

# Create the predicted tags: pred
pred = nb_tfidf_classifier.predict(tfidf_test)

# Calculate the accuracy score: score
score = metrics.accuracy_score(y_test, pred)
print(score)

# Calculate the confusion matrix: cm
cm = metrics.confusion_matrix(y_test, pred, labels=['entertainment', 'politics', 'business', 'sports', 'tech'])
print(cm)


0.961730449251248
[[111   2   3   5   4]
 [  0 105   1   0   0]
 [  0   1 132   0   2]
 [  0   0   0 128   0]
 [  0   1   3   1 102]]


#### Improving your model


In [49]:
# Create the list of alphas: alphas
alphas = np.arange(0, 1, .1)

# Define train_and_predict()
def train_and_predict(alpha):
    # Instantiate the classifier: nb_classifier
    nb_cv_lassifier = MultinomialNB(alpha=alpha)
    # Fit to the training data
    nb_cv_lassifier.fit(count_train, y_train)
    # Predict the labels: pred
    pred = nb_cv_lassifier.predict(count_test)
    # Compute accuracy: score
    score = metrics.accuracy_score(y_test, pred)
    return score

# Iterate over the alphas and print the corresponding score
best_alfa_score = {}
for alpha in alphas:
    best_alfa_score[alpha] = train_and_predict(alpha)
    print('Alpha: ', alpha)
    print('Score: ', train_and_predict(alpha))
    print()
best_score = max(best_alfa_score, key=best_alfa_score.get)

print("Best alfa: ", best_score, "best accuracy: ",best_alfa_score[best_score])

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha:  0.0
Score:  0.9733777038269551

Alpha:  0.1
Score:  0.9850249584026622

Alpha:  0.2
Score:  0.9850249584026622

Alpha:  0.30000000000000004
Score:  0.9883527454242929

Alpha:  0.4
Score:  0.9866888519134775

Alpha:  0.5
Score:  0.9866888519134775

Alpha:  0.6000000000000001
Score:  0.9866888519134775

Alpha:  0.7000000000000001
Score:  0.9833610648918469

Alpha:  0.8
Score:  0.9833610648918469

Alpha:  0.9
Score:  0.9816971713810316

Best alfa:  0.30000000000000004 best accuracy:  0.9883527454242929


In [50]:
# Create the list of alphas: alphas
alphas = np.arange(0, 1, .1)

# Define train_and_predict()
def train_and_predict(alpha):
    # Instantiate the classifier: nb_classifier
    nb_tfidf_classifier = MultinomialNB(alpha=alpha)
    # Fit to the training data
    nb_tfidf_classifier.fit(tfidf_train, y_train)
    # Predict the labels: pred
    pred = nb_tfidf_classifier.predict(tfidf_test)
    # Compute accuracy: score
    score = metrics.accuracy_score(y_test, pred)
    return score

# Iterate over the alphas and print the corresponding score
best_alfa_score = {}
for alpha in alphas:
    best_alfa_score[alpha] = train_and_predict(alpha)
    print('Alpha: ', alpha)
    print('Score: ', train_and_predict(alpha))
    print()
best_score = max(best_alfa_score, key=best_alfa_score.get)

print("Best alfa: ", best_score, "best accuracy: ",best_alfa_score[best_score])

/home/nbuser/anaconda3_420/lib/python3.5/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha:  0.0
Score:  0.9683860232945092

Alpha:  0.1
Score:  0.9850249584026622

Alpha:  0.2
Score:  0.9833610648918469

Alpha:  0.30000000000000004
Score:  0.978369384359401

Alpha:  0.4
Score:  0.9767054908485857

Alpha:  0.5
Score:  0.9750415973377704

Alpha:  0.6000000000000001
Score:  0.9717138103161398

Alpha:  0.7000000000000001
Score:  0.9717138103161398

Alpha:  0.8
Score:  0.9700499168053245

Alpha:  0.9
Score:  0.9650582362728786

Best alfa:  0.1 best accuracy:  0.9850249584026622


#### Inspecting the model


In [52]:
# Get the class labels: class_labels
class_labels = nb_classifier.classes_

# Extract the features: feature_names
feature_names = tfidf_vectorizer.get_feature_names()

# Zip the feature names together with the coefficient array and sort by weights: feat_with_weights
feat_with_weights = sorted(zip(nb_classifier.coef_[0], feature_names))

# Print the first class label and the top 20 feat_with_weights entries
print(class_labels[0], feat_with_weights[:20])

# Print the second class label and the bottom 20 feat_with_weights entries
print(class_labels[1], feat_with_weights[-20:])


business [(-10.448122083212702, '00'), (-10.448122083212702, '0001'), (-10.448122083212702, '00051'), (-10.448122083212702, '002'), (-10.448122083212702, '004secs'), (-10.448122083212702, '007'), (-10.448122083212702, '0100'), (-10.448122083212702, '0130'), (-10.448122083212702, '019secs'), (-10.448122083212702, '0227'), (-10.448122083212702, '028'), (-10.448122083212702, '030'), (-10.448122083212702, '0300'), (-10.448122083212702, '0305'), (-10.448122083212702, '0469'), (-10.448122083212702, '050505'), (-10.448122083212702, '053'), (-10.448122083212702, '053bn'), (-10.448122083212702, '0605'), (-10.448122083212702, '0700')]
entertainment [(-8.67461810361125, 'quarter'), (-8.651653481004878, 'new'), (-8.648752865809342, 'rate'), (-8.64696044438273, 'rise'), (-8.644716473046667, 'dollar'), (-8.587806528567564, 'government'), (-8.533053369227774, '2004'), (-8.473116806427768, 'firm'), (-8.467405603196003, 'shares'), (-8.413716338924312, 'sales'), (-8.402486142334473, 'company'), (-8.3763

### Test on new dataset

In [53]:
# Import new data
os.chdir('/home/nbuser/library/1. Classifier/3. Exploratory Data Analysis')
new_data = pd.read_csv('news_df_validation.csv')
new_data.head()

,file_name,title,news_text,category
0,214.txt,Mansfield 0-1 Leyton Orient,An second-half goal from Andy Scott condemned ...,sports
1,303.txt,Film production 'falls' 40% in UK,The number of British films produced in the UK...,entertainment
2,083.txt,Hague 'given up' his PM ambition,Former Conservative leader William Hague says ...,politics
3,190.txt,SA return to Mauritius,Top seeds South Africa return to the scene of ...,sports
4,103.txt,Minimum rate for foster parents,Foster carers are to be guaranteed a minimum a...,politics


#### Count Vectorizer on test dataset

In [58]:
def new_data_count_vec_predict(test_new_data):
    clean_df = clean_all_text(test_new_data)
    nb_cv_classifier = MultinomialNB(alpha=0.3)
    nb_cv_classifier.fit(count_train, y_train)
    count_test = count_vectorizer.transform(test_new_data['news_text'])
    new_cv_pred = nb_cv_classifier.predict(count_test)
    return new_cv_pred

In [59]:
def print_new_data_pred(test_data, index):
    new_data_cat = new_data['category']
    print(new_data_count_vec_predict(test_data)[index],' == ', new_data_cat[index])

In [60]:
print_new_data_pred(new_data, 28)

entertainment  ==  entertainment


In [61]:
count_vec_new_pred = new_data_count_vec_predict(new_data)

#### TFIDF on new dataset test

In [62]:
def new_data_ftidf_vec_predict(test_new_data):
    clean_df = clean_all_text(test_new_data)
    nb_tfidf_classifier = MultinomialNB(0.1)
    nb_tfidf_classifier.fit(tfidf_train, y_train)
    ftidf_test = tfidf_vectorizer.transform(test_new_data['news_text'])
    new_tfidf_pred = nb_tfidf_classifier.predict(ftidf_test)
    return new_tfidf_pred

In [63]:
tfidf_new_pred = new_data_ftidf_vec_predict(new_data)

In [64]:
print(tfidf_new_pred[112])

sports


### End the scooooore iiiiiiiis: 

In [65]:
count_vec_score = metrics.accuracy_score(count_vec_new_pred, new_data['category'])
print('Count vectorize score: ',round(count_vec_score, 4))
print()
tfidf_vec_score = metrics.accuracy_score(new_data_ftidf_vec_predict(new_data), new_data['category'])
print('TF-IDF score: ',round(tfidf_vec_score, 4))

Count vectorize score:  0.973

TF-IDF score:  0.964


### CV: 0.9775 <br><br>TFIDF: 0.973

### Saving models to pickle file

In [66]:
#Import pickle
import pickle

In [69]:
# save count_vector model to disk

os.chdir('/home/nbuser/library/1. Classifier/4. Feature Engineering')
filename = 'count_vec_model.pkl'
pickle.dump(nb_cv_classifier, open(filename, 'wb'))
 
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))

In [70]:
# save tf-idf_vector model to disk

filename = 'tfidf_vec_model.pkl'
pickle.dump(nb_tfidf_classifier, open(filename, 'wb'))
 
 

### Test on BBC live news

In [ ]:
#! run  '/home/nbuser/library/2. News Scrapper/BBC news scrapper with NewsApi.ipynb'

In [ ]:
BBC_live_news_df = pd.read_csv('/home/nbuser/library/2. News Scrapper/BBC_LIVE_NEWS.csv')

#### Predict live news class

In [ ]:
count_vec_live_pred = new_data_count_vec_predict(BBC_live_news_df)

In [ ]:
tfidf_live_pred = new_data_ftidf_vec_predict(BBC_live_news_df)

In [ ]:
BBC_live_news_df['CV Pred Category'] = count_vec_BBC_live_pred
BBC_live_news_df['TF-IDF Pred Category'] = tfidf_live_pred

In [ ]:
BBC_live_news_df.head()

#### Save the BBC live news prediction file

In [ ]:
def saving_file(file, file_name, save_dir):
    file.to_csv(os.path.join(save_dir,file_name), index=False)

In [ ]:
save_dir = '/home/nbuser/library/2. News Scrapper'
file = BBC_all_news_df
file_name = 'BBC_LIVE_NEWS_CLASSES.csv'

In [ ]:
saving_file(file, file_name, save_dir)